In [1]:
from heeps.config.read_config import read_config
from heeps.config.update_config import update_config
from heeps.util.img_processing import resize_cube
from astropy.io import fits
import numpy as np
import os

In [2]:
os.chdir(os.path.normpath(os.path.expandvars('$HOME/heeps_metis/input_files')))
npupil_L = update_config(**dict(read_config(), band='L'))['npupil']
npupil_N = update_config(**dict(read_config(), band='N2'))['npupil']
npupil = {'L':npupil_L, 'N2':npupil_N}
mask_L = fits.getdata('wavefront/cfull/mask_Cfull_20211007_%s_%s.fits'%('L', npupil_L)) > 0.5
mask_N = fits.getdata('wavefront/cfull/mask_Cfull_20211007_%s_%s.fits'%('N2', npupil_N)) > 0.5
mask = {'L':mask_L, 'N2':mask_N}
scao_L = fits.getdata('wavefront/cfull/cube_Cfull_20211007_3600s_300ms_0piston_meters_scao_only_%s_%s.fits'%('L', npupil_L))
scao_N = fits.getdata('wavefront/cfull/cube_Cfull_20211007_3600s_300ms_0piston_meters_scao_only_%s_%s.fits'%('N2', npupil_N))
scao = {'L':scao_L, 'N2':scao_N}
filename = 'wavefront/ncpa/%s/DIFF_rep_%s_field_%s.fits'
tag = {'L':'LM_20211122', 'N2':'N_20211122'}

In [3]:
for band, rep in zip(['L','N2'],[15,13]):
    start = 38 # 0, 13, 25, 32, (38), 44 
    for i,field in enumerate(range(start, start+25)):
        ncpa = fits.getdata(filename%(tag[band], rep, field))
        ncpa = resize_cube(ncpa, npupil[band])
        ncpa -= np.mean(ncpa[mask[band]]) # remove piston
        ncpa[~mask[band]] = 0             # mask with zeros
        stack = np.dstack([ncpa.T]*480).T
        if i == 0:
            cube = stack
        else:
            cube = np.vstack((cube, stack))
    fits.writeto('wavefront/ncpa/%s_%s_%s_rep_%s.fits'%(tag[band], band, npupil[band], rep), cube, overwrite=True)
    print('NCPA: %.2f nm'%(np.mean([np.std(x[mask[band]]) for x in cube*1e9])))
    cube += scao[band]
    fits.writeto('wavefront/cfull/cube_Cfull_20211007_3600s_300ms_0piston_meters_all_ncpa_%s_%s_rep_%s_-0h30.fits'%
                 (band, npupil[band], rep), cube, overwrite=True)

NCPA: 98.27 nm
NCPA: 95.44 nm
